In [76]:
#importy
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
from scipy import stats
import numpy as np

# Tworzenie tabeli z zebranymi ogłoszeniami i analiza danych.

In [77]:
def poczatek():
    url1 = input('Wprowadź link do strony z wynikami Twojego wyszukiwania. Przejdę przez wszystkie dostępne strony. ')
    strona = requests.get(url1)
    soup = bs(strona.text,'lxml')
    
    return soup, url1

#soup, url = poczatek()

In [78]:
soup, url1 = poczatek()

Wprowadź link do strony z wynikami Twojego wyszukiwania. Przejdę przez wszystkie dostępne strony. https://www.otodom.pl/wynajem/mieszkanie/tychy/?search%5Bfilter_float_m%3Ato%5D=50&search%5Bfilter_enum_rooms_num%5D%5B0%5D=2&search%5Bfilter_enum_rooms_num%5D%5B1%5D=3&search%5Bfilter_enum_rooms_num%5D%5B2%5D=1&search%5Bregion_id%5D=12&search%5Bsubregion_id%5D=294&search%5Bcity_id%5D=166


In [79]:
def zakres():    
    p2 = soup.find('form', {"id":'pagerForm'})
    p2 = p2.find_all('a',"")

    strony = []

    for a in p2:
        strona = a.text
        if strona != "":
            strona = int(strona)
            strony.append(strona)
        else:
            pass
    strona_max = max(strony)
    strona_min = min(strony)

    return(strona_min, strona_max)

#strona_min, strona_max = zakres()

In [80]:
strona_min,strona_max = zakres()

In [81]:
def limit_wyszukiwan(strona_max):

    limit_wyszukiwania = 0
    
    if strona_max > 50:
        limit_wyszukiwania =+ 50
    else:
        limit_wyszukiwania =+ strona_max
    
    return limit_wyszukiwania

In [82]:
limit_wyszukiwania = limit_wyszukiwan(strona_max)
limit_wyszukiwania

2

In [104]:
def parametry():
    
    #tworze główny data frame
    df = pd.DataFrame(columns=['tytul','typ','miejsce','pokoje','pietro','pietra_budynku','rok_budowy',
                               'powierzchnia','cena',
                               'czynsz_osobny','kaucja','wynajmujacy','link'])
    
    #listy na wartości z ogłoszeń + tworzenie kolejnych iteracji linku
    for i in range(strona_min,limit_wyszukiwania+1):
        
        l_tytul = []
        l_typ = []
        l_miejsce = []
        l_pokoje = []
        l_powierzchnia = []
        l_cena = []
        l_wynajmujacy = []
        
        
        link_oferta = []
        
        l_czynsz_dod = []
        l_kaucja =[]
        l_pietro = []
        l_pietra = []
        l_rok_bud = []
        
        url = url1+ f'&page={i}'
        strona = requests.get(url)
        soup = bs(strona.text,'lxml')

        oferty = soup.find_all('article')
    
        #pętla ściągania ofert z pojedynczej strony wyszukiwania
        for oferta in oferty:

            tytul = oferta.find('span', {"class":"offer-item-title"}).text
            l_tytul.append(tytul)

            typ = oferta.find('span', {"class": "hidden-xs"}).text.strip().rstrip(":")
            l_typ.append(typ)

            miejsce = oferta.find('p', {"class": "text-nowrap"}).text
            miejsce = miejsce[miejsce.find(":",0,-1)+2:]
            l_miejsce.append(miejsce)

            pokoje = int(oferta.find('li', {'class': 'offer-item-rooms hidden-xs'}).text[0])
            l_pokoje.append(pokoje)

            powierzchnia = int(re.findall(r'\d+',oferta.find('li', {"class": "hidden-xs offer-item-area"}).text)[0])
            l_powierzchnia.append(powierzchnia)

            cena = int(re.findall(r'\d+',re.sub(r'\s+','',oferta.find('li', {'class': "offer-item-price"}).text.strip()))[0])
            l_cena.append(cena)
            
            #wynajmujacy - rozdzielenie osoby prywatnej i pośrednika
            wynajmujacy = oferta.find('ul', {'class': "params-small clearfix hidden-xs"}).text.strip()
            if wynajmujacy == 'Oferta prywatna':
                l_wynajmujacy.append(wynajmujacy)
            else:
                l_wynajmujacy.append('posrednik')
            
            #url do oferty
            url2 = oferta.find_all('a')[0].get('href')
            link_oferta.append(url2)
            
            #pobieranie danych dodatkowych z strony konkretnego ogłoszenia.
            strona2 = requests.get(url2)
            soup2 = bs(strona2.text,'lxml')

            try:
                pietro = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Piętro'})[0].text)
                l_pietro.append(pietro[0])
            except:
                l_pietro.append(0)

            try:
                czynsz_dod = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Czynsz - dodatkowo'})[0].text)
                l_czynsz_dod.append(czynsz_dod[0])
            except:
                l_czynsz_dod.append(0)

            try:
                kaucja = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Kaucja'})[0].text.replace(" ",""))
                l_kaucja.append(kaucja[0])    
            except:
                l_kaucja.append(0)

            try:
                rok_budowy = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Rok budowy'})[0].text)
                l_rok_bud.append(rok_budowy[0])
            except:
                l_rok_bud.append("")

            try:
                liczba_pieter = re.findall(r'\d+',soup2.find_all('div', {'aria-label': 'Liczba pięter'})[0].text)
                l_pietra.append(liczba_pieter[0])
            except:
                l_pietra.append("")

            
        #tworze data frame z jednej strony wyszukiwania
        df1 = pd.DataFrame({'tytul':l_tytul, 
                            "typ":l_typ,
                            "miejsce": l_miejsce,
                            "pokoje": l_pokoje,
                            "pietro": l_pietro,
                            "pietra_budynku": l_pietra,
                            "rok_budowy": l_rok_bud,
                            "powierzchnia": l_powierzchnia,
                            "cena": l_cena,
                            "czynsz_osobny": l_czynsz_dod,
                            "kaucja": l_kaucja,
                            "wynajmujacy": l_wynajmujacy,
                            "link": link_oferta}) 

        
        #dodaję df z jednej strony wyszukiwania do df głównego
        ramki = [df,df1]
        df = pd.concat(ramki)
    
    
    #dopasowuje automatycznie typy danych
    df = df.convert_dtypes()
    
    #wyciągam miasto z tekstu
    df['miasto'] = df['miejsce'].str.split(',').str[0]
    
    #dodaje typy danych kategoryczne + float
    df['miasto'] = df['miasto'].astype('category')
    df['typ'] = df['typ'].astype('category')
    df['miejsce'] = df['miejsce'].astype('category')
    df['wynajmujacy'] = df['wynajmujacy'].astype('category')
    df['cena'] = df['cena'].astype('int')
    df['czynsz_osobny']= df['czynsz_osobny'].astype('int')
    df['kaucja'] = df['kaucja'].astype('int')
    
    #dodaję sumy dla opłat
    df['suma_oplat'] = df[['cena','czynsz_osobny']].sum(axis = 1)
    df['pierwsza_oplata'] = df[['cena','czynsz_osobny','kaucja']].sum(axis = 1)
    
    #zmiana kolejności kolumn
    df = df[['tytul','typ','miasto','miejsce','pokoje','pietro','pietra_budynku','rok_budowy',
             'powierzchnia','cena','czynsz_osobny','kaucja','suma_oplat','pierwsza_oplata',
             'wynajmujacy','link']]
    
    #usuwanie duplikatów wierszy
    df = df.drop_duplicates()
    df = df.reset_index(drop = True)
    
    #usuwanie wartości bardzo odstających z ceny
    df['z_score'] = np.abs(stats.zscore(df['cena'])<3)
    df = df[df['z_score']==True]
    
    return df

In [105]:
tabela = parametry()

In [106]:
tabela

,tytul,typ,miasto,miejsce,pokoje,pietro,pietra_budynku,rok_budowy,powierzchnia,cena,czynsz_osobny,kaucja,suma_oplat,pierwsza_oplata,wynajmujacy,link,z_score
0,Wynajmę bezpośrednio,Mieszkanie na wynajem,Tychy,"Tychy, śląskie",2,3,10,1977,36,1200,380,1500,1580,3080,posrednik,https://www.otodom.pl/pl/oferta/wynajme-bezpos...,True
1,Wynajmę eleganckie mieszkanie - zarezerwowane,Mieszkanie na wynajem,Tychy,"Tychy, C",2,0,4,,39,1500,310,3200,1810,5010,posrednik,https://www.otodom.pl/pl/oferta/wynajme-elegan...,True
2,"TYCHY - najpiękniejsze, reprezentacyjne mieszk...",Mieszkanie na wynajem,Tychy,"Tychy, śląskie",2,4,,,42,2100,1,3000,2101,5101,Oferta prywatna,https://www.otodom.pl/pl/oferta/tychy-najpiekn...,True
3,Nowe 2 pokoje z Miejscem w Hali Garażowej,Mieszkanie na wynajem,Tychy,"Tychy, śląskie",2,1,,,42,1700,0,0,1700,1700,posrednik,https://www.otodom.pl/pl/oferta/nowe-2-pokoje-...,True
4,Tychy ul. Biblioteczna,Mieszkanie na wynajem,Tychy,"Tychy, śląskie",2,1,3,1970,50,1200,0,0,1200,1200,posrednik,https://www.otodom.pl/pl/oferta/tychy-ul-bibli...,True
5,Nowoczesne mieszkanie - Strefa Ekonomiczna Tychy,Mieszkanie na wynajem,Tychy,"Tychy, śląskie",2,1,4,,39,1600,350,3000,1950,4950,Oferta prywatna,https://www.otodom.pl/pl/oferta/nowoczesne-mie...,True
6,"bez pośredników - BIERUŃ przy rynku, wyposażone",Mieszkanie na wynajem,Tychy,"Tychy, śląskie",1,0,2,,31,1300,0,1300,1300,2600,Oferta prywatna,https://www.otodom.pl/pl/oferta/bez-posredniko...,True
7,Mieszkanie 2 pokojowe z balkonem,Mieszkanie na wynajem,Tychy,"Tychy, Śródmieście",2,4,11,,43,1590,0,1700,1590,3290,posrednik,https://www.otodom.pl/pl/oferta/mieszkanie-2-p...,True
8,Wynajem Tychy osiedle Z,Mieszkanie na wynajem,Tychy,"Tychy, Paprocany",2,4,4,,47,1400,450,2000,1850,3850,posrednik,https://www.otodom.pl/pl/oferta/wynajem-tychy-...,True
9,Tychy Żwaków od natychmiast,Mieszkanie na wynajem,Tychy,"Tychy, Żwaków",2,3,4,2020,40,2000,300,2000,2300,4300,posrednik,https://www.otodom.pl/pl/oferta/tychy-zwakow-o...,True


In [86]:
tabela.dtypes

tytul                string
typ                category
miasto             category
miejsce            category
pokoje                Int64
pietro               object
pietra_budynku       string
rok_budowy           object
powierzchnia          Int64
cena                  int32
czynsz_osobny         int32
kaucja                int32
suma_oplat            int64
pierwsza_oplata       int64
wynajmujacy        category
link                 string
z_score                bool
dtype: object

# Analiza i tabele

In [87]:
#importy bibliotek
import plotly.express as px


In [88]:
#podsumowanie zebranych ofert

def podsumowanie_ofert():
    
    #podsu = tabela.groupby(['miasto','pokoje']).count().reset_index()
    
    podsu = tabela.groupby(['miasto','pokoje']).agg({'cena':['mean', 'max', 'min'],
                         'suma_oplat':['mean','max','min'],
                         'powierzchnia':'mean', 
                         'tytul':'count'})
    
    print("Podsumowanie ofert z Twojego wyszukiwania OTODOM: ")
    
    return podsu

In [89]:
podsumowanie = podsumowanie_ofert()
podsumowanie

Podsumowanie ofert z Twojego wyszukiwania OTODOM: 


cena               suma_oplat             powierzchnia  \
                      mean   max   min         mean   max   min         mean   
miasto pokoje                                                                  
Tychy  1       1371.666667  1590  1200  1385.000000  1590  1200    31.444444   
       2       1663.589744  3000  1200  1784.923077  3000  1200    41.153846   
       3        600.000000   600   600   600.000000   600   600    42.000000   

              tytul  
              count  
miasto pokoje        
Tychy  1          9  
       2         39  
       3          1

## Wykresy

In [90]:
oferty_miasto = tabela.groupby('miasto').count()
oferty_miasto['liczba_ofert'] = oferty_miasto['tytul']
oferty_miasto = oferty_miasto.reset_index()[['miasto','tytul']]
oferty_miasto.columns = ['miasto','liczba_ofert']
oferty_miasto

of_miasto_wykres = px.pie(oferty_miasto,
                         values = 'liczba_ofert',
                         names = 'miasto',
                         title = "Liczba ofert w danym mieście."
                         )

of_miasto_wykres

In [91]:
powierzchnia_miasto = px.bar(tabela,
                      y = 'powierzchnia',
                      x = 'miasto',
                      title = 'Dostępna powierzchnia z Twojego wyszukiwania wg. miasta. [m2]',
                      )
powierzchnia_miasto.show()

In [92]:
pokoje_box = px.box(tabela, 
                    y = 'cena',
                    points = 'all',
                    hover_data = ['powierzchnia','pokoje','miasto'],
                    title = 'Rozrzut wszystkich ofert wg. ceny.- przybliż wykres w interesujące miejsce',
                    
                   )
pokoje_box

In [93]:
pokoje_suma_oplat = px.box(tabela, 
                    y = 'suma_oplat',
                    points = 'all',
                    hover_data = ['powierzchnia','pokoje','miasto'],
                    title = 'Rozrzut wszystkich ofert wg. sumy opłat msc.',
                    
                   )
pokoje_suma_oplat

In [94]:
powierzchnia_cena_scatter = px.scatter(tabela,
                           y = 'cena',
                           x = 'powierzchnia',
                           title = 'Rozrzut cenowy ofert.', 
                           color = 'miasto')

powierzchnia_cena_scatter

In [95]:
cena_pokoje_box = px.box(tabela,
                                y = 'cena',
                                x = 'pokoje',
                                color = 'miasto',
                                title = 'Rozrzut cenowy ofert (wg l.pokojów)')

cena_pokoje_box

In [96]:


suma_oplat_box = px.box(tabela,
                                y = 'suma_oplat',
                                x = 'pokoje',
                                color = 'miasto',
                                title = 'Rozrzut cenowy pełnej opłaty per msc (wg l.pokojów)')

suma_oplat_box

In [97]:
powierzchnia_suma_oplat_scatter = px.scatter(tabela,
                           y = 'suma_oplat',
                           x = 'powierzchnia',
                           title = 'Rozrzut sum oplat za mieszkanie.', 
                           color = 'miasto')

powierzchnia_suma_oplat_scatter

In [98]:
tabela_kaucja = tabela[tabela['kaucja']>0]

kaucja_box = px.box(tabela_kaucja,
                                y = 'kaucja',
                                x = 'pokoje',
                                color = 'miasto',
                                title = 'Rozrzut cenowy kaucji wstepnej')

kaucja_box

In [99]:
pierwsza_oplata_box = px.box(tabela,
                                y = 'pierwsza_oplata',
                                x = 'pokoje',
                                color = 'miasto',
                                title = 'Rozrzut cenowy pierwszej oplaty')

pierwsza_oplata_box

In [100]:
kaucja_box_wynajmujacy = px.box(tabela_kaucja,
                                y = 'kaucja',
                                x = 'pokoje',
                                color = 'wynajmujacy',
                                title = 'Rozrzut cenowy kaucji wstepnej wg. wynajmującego')

kaucja_box_wynajmujacy

In [101]:
czynsz_dodatkowy = tabela[tabela['czynsz_osobny']>10]

czynsz_osobny = px.box(czynsz_dodatkowy,
                                y = 'czynsz_osobny',
                                x = 'pokoje',
                                color = 'miasto',
                                title = 'Rozrzut czynszu (nie wliczonego do ceny z ogloszenia)')

czynsz_osobny

In [102]:
czynsz_osobny_wynajmujacy = px.box(czynsz_dodatkowy,
                                y = 'czynsz_osobny',
                                x = 'pokoje',
                                color = 'wynajmujacy',
                                title = 'Rozrzut czynszu (nie wliczonego do ceny z ogloszenia)')

czynsz_osobny_wynajmujacy

In [103]:
oferty_z_dodat_czynszem = tabela[tabela['czynsz_osobny']>0]
print(f'Ogłoszenia które na stronie głównej podają cenę bez czynszu: {len(oferty_z_dodat_czynszem)}')

oferty_czynsz_wliczony = tabela[tabela['czynsz_osobny']==0]
print(f'Ogłoszenia które podają cenę z już wliczonym czynszem: {len(oferty_czynsz_wliczony)}')

Ogłoszenia które na stronie głównej podają cenę bez czynszu: 17
Ogłoszenia które podają cenę z już wliczonym czynszem: 32
